In [1]:
from os.path import join, basename, dirname, abspath
import os
from glob import glob
import pandas as pd
import numpy as np

In [2]:
MODELS_DIR = '../models/'

In [3]:
results = [[*x.split('/')[-2].split('-'), basename(x).split(':')[1]] 
           for x in glob(join(MODELS_DIR, '*folds*/roc-auc:*'))]

results = [[result[0][:12], result[0][13:], result[1], result[2][0], 
            result[2][2:-5].rstrip('_'), result[2][-5:].lstrip('_'), result[3]] 
            for result in results]

results.extend([['exp_train_02', np.nan, np.nan, np.nan, np.nan, np.nan, '0.8263'], 
                ['exp_train_03', np.nan, np.nan, np.nan, np.nan, np.nan, '0.7252'], 
                ['exp_train_04', np.nan, np.nan, np.nan, np.nan, np.nan, '0.8620'], 
                ['exp_train_05', np.nan, np.nan, np.nan, np.nan, np.nan, '0.8892'], 
                ['exp_train_06', np.nan, np.nan, np.nan, np.nan, np.nan, '0.9040'], 
                ['exp_train_07', np.nan, np.nan, np.nan, np.nan, np.nan, '0.8529']])

In [4]:
results_df = pd.DataFrame(results, 
                          columns=['Experiment', 'Pseudo Label Type', 'Pct', 'Smooth Func Code', 
                                   'Folds', 'Fine-Tune Only Last Layer', 'Hold-Out AUC-ROC'])
results_df['Hold-Out AUC-ROC'] = results_df['Hold-Out AUC-ROC'].apply(float)
print(results_df.shape)
results_df.sample(5)

(111, 7)


,Experiment,Pseudo Label Type,Pct,Smooth Func Code,Folds,Fine-Tune Only Last Layer,Hold-Out AUC-ROC
71,exp_train_02,rnd_undspml,80,1,folds_stratified,false,0.7837
13,exp_train_02,rnd_undspml,80,6,folds_stratified,false,0.7612
2,exp_train_02,rdf_bnd,50,4,folds_stratified,false,0.8153
95,exp_train_02,rdf_bnd,100,2,folds_stratified,true,0.8129
40,exp_train_02,rdf_bnd,100,4,folds_stratified,true,0.8307


In [5]:
results_df.to_csv('../input/experiment_results.csv', index=False)
results_df = results_df[results_df['Experiment'].apply(lambda x: 'exp_train_02' in x)]
print(results_df.shape)

(106, 7)


In [6]:
results_df[pd.isna(results_df['Pseudo Label Type'])]

,Experiment,Pseudo Label Type,Pct,Smooth Func Code,Folds,Fine-Tune Only Last Layer,Hold-Out AUC-ROC
105,exp_train_02,NaN,NaN,NaN,NaN,NaN,0.8263


In [7]:
results_df[(results_df['Pct'] == '100') & (results_df['Smooth Func Code'] == '1')]

,Experiment,Pseudo Label Type,Pct,Smooth Func Code,Folds,Fine-Tune Only Last Layer,Hold-Out AUC-ROC
36,exp_train_02,rdf_bnd,100,1,folds_stratified,true,0.8313
48,exp_train_02,str_undspml,100,1,folds_stratified,true,0.8259
65,exp_train_02,rnd_undspml,100,1,folds_stratified,true,0.8387


In [8]:
results_df.groupby(['Fine-Tune Only Last Layer'])['Hold-Out AUC-ROC'].mean()

Fine-Tune Only Last Layer
false    0.771188
true     0.833531
Name: Hold-Out AUC-ROC, dtype: float64

In [9]:
results_df = results_df[results_df['Fine-Tune Only Last Layer'] == 'true']

In [10]:
results_df.groupby(['Pseudo Label Type'])['Hold-Out AUC-ROC'].mean().sort_values()

Pseudo Label Type
str_undspml    0.829139
rnd_undspml    0.832778
rdf_bnd        0.838678
Name: Hold-Out AUC-ROC, dtype: float64

In [11]:
results_df.groupby(['Pct'])['Hold-Out AUC-ROC'].mean().sort_values()

Pct
100    0.800089
80     0.848506
50     0.852000
Name: Hold-Out AUC-ROC, dtype: float64

In [12]:
results_df.groupby(['Smooth Func Code'])['Hold-Out AUC-ROC'].mean().sort_values()

Smooth Func Code
6    0.827533
2    0.828844
5    0.831044
3    0.834544
4    0.835289
1    0.843933
Name: Hold-Out AUC-ROC, dtype: float64